# Synthetic EEG-Data and the Forward Solution with MNE

In this tutorial, we explore how we can simulate synthetic EEG data. Personally, I think this is a really exciting topic since it boils down to understanding brain physics. Furthermore, if one is able to simulate meaningful EEG data, which renders cognitive processes, this would mean a large step in this field. However, this is not our aim for the seminar (but could be interesting for a master thesis topic ;) ). 

Obviously, the simulation of artificial EEG data allows us to provide a large corpus of data for deep neural networks without executing costly experiments. In the context of our projects, this is particularly interesting for the source reconstruction task, but also for the unsupervised feature learning. For the latter one, you could explore if you can learn representations from synthetic data, which could be kind of a proof of concept for reliable data generation. For a more detailed treatment of this topic, you can look into the tutorials and explanations from the MNE page, linked below, as well as in the book from Nunez, linked in the first tutorial.
- [Head model and forward compuatation](https://mne.tools/stable/auto_tutorials/forward/30_forward.html#sphx-glr-auto-tutorials-forward-30-forward-py)
- [EEG forward operator with a template MRI
](https://mne.tools/stable/auto_tutorials/forward/35_eeg_no_mri.html)
- [https://mne.tools/stable/auto_tutorials/inverse/10_stc_class.html](https://mne.tools/stable/auto_tutorials/inverse/10_stc_class.html): The source estimate class, which holds the source space with its activations (the active brain)
- [Implementation details](https://mne.tools/stable/overview/implementation.html): Here, you can find information on head models, the boundary element method and more. 

If you want a slightly shorter summary of the topic, compared to the Nunez book, you can take a look into [this paper](https://jneuroengrehab.biomedcentral.com/articles/10.1186/1743-0003-4-46). Heavily related to this topic is the solution to the inverse problem of EEG. By this, we mean the process of inferring the sources within the brain which caused the measured EEG signal. 

Basically, this tutorial has two parts, which are put togehter from the MNE webpage. On the one hand, we need a forward solution. On the other hand, we need predefined source space activities, so that we can compute the EEG signals from it. In the end, as a little bonus, we will look at how we can produce datasets in braindecode from raw data structures. 



In [ ]:
# Install MNE from GitHub
!pip --quiet install mne

In [ ]:
# Importing libraries
import os
import os.path as op
import numpy as np
import matplotlib.pyplot as plt

# Import MNE
import mne
from mne.datasets import sample
from mne.datasets import eegbci
from mne.datasets import fetch_fsaverage

## Preparation of Data Structures and Data
To create synthetic EEG data, we will use different MNE facilities, namely
- the Raw data structure,
- the Montage data structure,
- the Info data structure,
- the Source Space (src) data structure,
- the Source Time Course (stc) data structure,
- the Forward data structure

The Raw and the Info structure is already known to us. The remaining ones will be explained below, when we go through the code. 

We will start with the preparation of the raw file

In [ ]:
# At first, we load a montage. This is important, since we need to know the sensor
# positions as well as the number of electrodes. Here we take the standard 1005
# system. The electrodes are placed in a 10% or 5% distance. However, we do not
# want to work with the complete set of electrodes, which is why we specify certain 
# electrodes
montage = mne.channels.make_standard_montage('standard_1005')
ch_names = [
            "Fp1", "F3", "F7", "FT9", "FC5", "FC1", "C3", "T7", "TP9", "CP5",
            "CP1", "Pz", "P3", "P7", "O1", "Oz", "O2", "P4", "P8", "TP10", 
            "CP6", "CP2", "Cz", "C4", "T8", "FT10", "FC6", "FC2", "F4", "F8", 
            "Fp2"
            ]

# We want to produce EEG signals only, so that we set the channel types accordingly
ch_types = ["eeg" for name in ch_names]

# Set a sampling frequency
sampling_freq = 200 

# Create info object
info = mne.create_info(ch_names=ch_names, sfreq=sampling_freq, ch_types=ch_types)

# Dummy data, specifies the number of time steps later
t = np.arange(0, 100, 1/sampling_freq)
freqs = [i*0.1 for i in range(len(ch_types))]
data = np.array([np.sin(freq*t*np.pi)  for freq in freqs])

# We can create a raw object from the RawArray data structure, rather than 
# instanciating a raw object directly
raw = mne.io.RawArray(data, info)

# Plot dummy data
raw.plot(show_scrollbars=False, show_scalebars=False, scalings=1.1)

# Set the montage
raw.set_montage(montage=montage)
raw.pick_channels(ch_names=ch_names)  # Channel names nochmal auswählen

## Part I - The Forward Solution

The first part deals with the calculation of the so called forward model. The idea is, that you can **calculate a linear transformation**, represented through a matrix, that **maps the source activity** in the brain **to the EEG sensors** and gives us the **desired EEG signal**. Therefore, we need a discrete version of our brain and our head! As you read this, you can already imagine, that this is non-trivial. I do not want to go into the details here as much as the topic is it worth, however, I want to give a short overview.

As already mentioned in the first tutorial (did I?), there are **various sources in the brain**, which contribute to the overall EEG signal. It is nearly a hopeless task to represent all of them by their single contribution. A successful approach to overcome this, is the so called **equivalent current dipole (ECD)** model. This model represent the activities of the different sources patch-wise with a single dipole. This makes things easier for us. From this point on, we have to make different assumptions, about how many boundaries are separating the source from the electrodes, the physical properties of those boundaries and so on. This leads one to the question, **how we describe our head and the brain** to model this process. Below, you can see a picture of how such a model looks like. The most common approaches are the Finite-Element-Method (FEM) as well as the **Boundary-Element-Method (BEM)**. The geometry depicted below corresponds to the latter one. 

<div>
<img src="https://drive.google.com/uc?export=view&id=1XxusWfiGTylcfsLhaAkOqmLXdY4bET-R" width="850"/>

<div text-align=center class="caption">A 4-shell head model created from MRI data [1] </div>
</div>

In the picture, the **4 shells** represent the **brain** itself, the **cerebrospinal fluid (CSF)**, the **inner skull** and the **outer skull**. All these different shells have different **conductivity values** (how good they transfer electrical fields) and are anisotropic (that means, that the properties of the material depend on the direction). 

As you can see, the model consits of a large number of small, triangular elements. Those elements discretize our continuous space and thus, the governing equations (I spare you the math here). If you discretize the partial differential equations describing the shells, you end up with linear transformations and, luckily, the composition of linear transformations is linear again. In this way, we end up with a simple equation that describes our linear mapping through
\begin{equation}
E = LD + N, 
\end{equation}

where E is our EEG - signal for one time step, L is called the Leadfield matrix and D is the vector of activations in the brain. $N \sim \mathcal{N}(0, 1)$, is a vector of random noise, which can be added (the noise can be more complicated than a gaussian distribution)

---

We will close the theory here. If you want to know more, look into the sources below. We will now start with the implementation now and look at the different aspects. 

[1] Miklody, Daniel: "Theory and Application for Spontaneous EEG", Phd Thesis, TU Berlin, 2015

[2] Nunez, Paul L.; Srinivasan, Ramesh: "Electric Fields of the Brain: The neurophysics of EEG", Oxford University Press, 2008

[3] Hallez, Hans et. al.: Review on solving the forward problem in EEG source analysis, Journal of NeuroEngineering and Rehabilitation, 2008

In [ ]:
# Download fsaverage files 
print("Fetch fsaverage data ... ", end="")
fs_dir = fetch_fsaverage(verbose=False)     
subjects_dir = op.dirname(fs_dir)
print("Done.")

# Specify the subject (tells MNE something about the subjects head)
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has an inbuild fsaverage Transformation

# Bem model
print("Create BEM model ... ", end="")
surf = mne.make_bem_model(subject=subject, ico=4, conductivity=[0.3, 0.006, 0.3], subjects_dir=subjects_dir, verbose=False)
print("Done.")
print("Compute solution of BEM model ... ", end="")
bem_model = mne.make_bem_solution(surfs=surf, verbose=False)
print("Done.")
plot_bem_kwargs = dict(
    subject=subject, subjects_dir=subjects_dir,
    brain_surfaces='white', orientation='coronal',
    slices=[50, 100, 150, 200])

print("\nPlot BEM model\n"
      "--------------")
mne.viz.plot_bem(**plot_bem_kwargs)

# Creating the source space
# The source space is, what we call our brain. In the end it is a tesselation of 
# the brain structure which yields vertices and triangular patches. This allows
# the calculation of activities at some point. The spacing is the mesh resolution.
# The distance measure is patch (triangle) based
print("Create Source space ...", end="")
src = mne.setup_source_space(
    subject, spacing='oct4', add_dist='patch', subjects_dir=subjects_dir, verbose=False
    )

print(src)
print("Done")

print("\nPlot BEM model with src\n"
      "--------------")
mne.viz.plot_bem(src=src, **plot_bem_kwargs)

# Solve the problem --> Leadfield
print("Solve the forward problem ...", end="")
fwd = mne.make_forward_solution(
    raw.info, trans=trans, src=src, bem=bem_model, eeg=True, mindist=5.0, n_jobs=1, verbose=False
    )

print(fwd["sol"]["data"])


# We transform the solution such that it is orthogonal to the head / brain 
fwd_fixed = mne.convert_forward_solution(
    fwd, surf_ori=True, force_fixed=True, use_cps=True, verbose=False
    )
print("Done.")

## Part II - Simulating EEG from a sparse Source Time Course

The second part is needed, to fill our source space with activations. We will create a sparsely populated source space. That means, we randomly select a small number of dipoles from the available source space and assign activations to them. This results in pointwise activations. Obviously, this is not realistic at all. Nevertheless, it serves as a good starting point for your journey. 

At the top of the tutorial we created the Raw data array with a dummy info object and a montage of our choice, where we picked a subset of EEG electrodes. This Raw array will be populated from the solution in the end. 

In [ ]:
# Setting the frame
n_dipoles = 10  # how many active dipols shall be distributed
duration = 2  # how long an event takes place
n = 0  # dummy variable to create different activation functions
rng = np.random.RandomState(10)  # Gives reproducible results

def wave_fun(times):
  global n

  n_samples = len(times)

  # Create a window where activation takes place
  window = np.zeros(n_samples)
  
  # Start and stop time of the activation
  start, stop = [int(ii * float(n_samples) /(2 * n_dipoles)) for ii in (2*n, 2*n+1)]
  print(f"Start: {start}, Stop: {stop}")

  # Set the window to one in the start-stop frame
  window[start:stop] = 1.
  n += 1

  # Create sine activation
  data = 25e-9 * np.sin(2. * np.pi * 10. * n * times)  # scale the data to nano-volt (usually employed for sources)
  data *= window

  return data


In [ ]:
""" We can simulate the source data now and inspect it """

# Time steps
times = raw.times[:int(raw.info['sfreq'] * duration)]
print(f"t_max: {times[-1]}")

# Simulate a sparse source space
stc = mne.simulation.simulate_sparse_stc(
    src, n_dipoles=n_dipoles, times=times, data_fun=wave_fun, random_state=rng
    )

# look at our source data
fig, ax = plt.subplots(1)
ax.plot(times, 1e9 * stc.data.T)
ax.set(ylabel='Amplitude (nAm)', xlabel='Time (s)')
mne.viz.utils.plt_show()

In [ ]:
"""
It is quite easy to simulate the EEG data now.
We simply apply an inbuild mne method, where we pass all the raw.info stuff we need. 
Furthermore
"""
raw = mne.simulation.simulate_raw(
    raw.info, [stc] * 10, forward=fwd, verbose=True
    )

raw.plot(show_scalebars=False, show_scrollbars=False)

# We can add noise to our data
cov = mne.make_ad_hoc_cov(raw.info)
mne.simulation.add_noise(
    raw, cov, iir_filter=[0.2, -0.2, 0.04], random_state=rng
    )

raw.plot(show_scalebars=False, show_scrollbars=False)